In [1]:
!nvidia-smi

Thu Feb 17 14:59:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8     2W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision, AUC, binary_accuracy, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from helper_functions import mcc_metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from transformers import AutoTokenizer

In [3]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [4]:
data_new = data.dropna(subset=['IUPAC_name'])
X = data_new['IUPAC_name']
target=data_new['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('gumgo91/IUPAC_BERT', from_pt = True)

In [6]:
iupac_data = tokenizer(list(X),  truncation=True, padding=True, max_length=256)
iupac_data = iupac_data['input_ids']

In [7]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(iupac_data, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [8]:
len(y_classification)

6170

In [9]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([3992, 2178], dtype=int64))

In [10]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 1 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [11]:
word_in_vocab = tokenizer.vocab
len(word_in_vocab)

1500

In [12]:
np.array(xtrain[0]).shape

(256,)

In [13]:
input = Input(shape = (256,))

x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=256, 
                        input_length=None,
                        )(input)
x = Bidirectional(LSTM(128, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(128, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(128, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [14]:
class_weights

array([0.77582335, 1.4063772 ])

In [15]:
class_weights = {0: 0.77582335,
                1: 1.4063772}
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'iupac'+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(34), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives()])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)

Epoch 1/60
130/130 [==============================] - ETA: 0s - loss: 0.6441 - binary_accuracy: 0.6889 - auc: 0.6745 - mcc_metric: nan - precision: 0.5970 - true_negatives: 2290.0000 - true_positives: 566.0000 - false_negatives: 908.0000 - false_positives: 382.0000
Epoch 00001: val_auc improved from -inf to 0.75479, saving model to F:/bbb/model/nlp\model_34iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


130/130 [==============================] - 51s 325ms/step - loss: 0.6441 - binary_accuracy: 0.6889 - auc: 0.6745 - mcc_metric: nan - precision: 0.5970 - true_negatives: 2290.0000 - true_positives: 566.0000 - false_negatives: 908.0000 - false_positives: 382.0000 - val_loss: 0.5517 - val_binary_accuracy: 0.7747 - val_auc: 0.7548 - val_mcc_metric: 0.4864 - val_precision: 0.6923 - val_true_negatives: 450.0000 - val_true_positives: 162.0000 - val_false_negatives: 106.0000 - val_false_positives: 72.0000
Epoch 2/60
130/130 [==============================] - ETA: 0s - loss: 0.5135 - binary_accuracy: 0.7877 - auc: 0.8110 - mcc_metric: 0.5298 - precision: 0.7407 - true_negatives: 2352.0000 - true_positives: 914.0000 - false_negatives: 560.0000 - false_positives: 320.0000
Epoch 00002: val_auc improved from 0.75479 to 0.85685, saving model to F:/bbb/model/nlp\model_34iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


...

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


130/130 [==============================] - 42s 326ms/step - loss: 0.3145 - binary_accuracy: 0.8717 - auc: 0.9389 - mcc_metric: 0.7331 - precision: 0.7814 - true_negatives: 2306.0000 - true_positives: 1308.0000 - false_negatives: 166.0000 - false_positives: 366.0000 - val_loss: 0.4243 - val_binary_accuracy: 0.8114 - val_auc: 0.9081 - val_mcc_metric: 0.6275 - val_precision: 0.6676 - val_true_negatives: 404.0000 - val_true_positives: 237.0000 - val_false_negatives: 31.0000 - val_false_positives: 118.0000
Epoch 7/60
130/130 [==============================] - ETA: 0s - loss: 0.3100 - binary_accuracy: 0.8695 - auc: 0.9401 - mcc_metric: 0.7371 - precision: 0.7755 - true_negatives: 2292.0000 - true_positives: 1313.0000 - false_negatives: 161.0000 - false_positives: 380.0000
Epoch 00007: val_auc improved from 0.90808 to 0.91032, saving model to F:/bbb/model/nlp\model_34iupac.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_34iupac.tf\assets


130/130 [==============================] - 45s 347ms/step - loss: 0.3100 - binary_accuracy: 0.8695 - auc: 0.9401 - mcc_metric: 0.7371 - precision: 0.7755 - true_negatives: 2292.0000 - true_positives: 1313.0000 - false_negatives: 161.0000 - false_positives: 380.0000 - val_loss: 0.3721 - val_binary_accuracy: 0.8367 - val_auc: 0.9103 - val_mcc_metric: 0.6496 - val_precision: 0.7324 - val_true_negatives: 442.0000 - val_true_positives: 219.0000 - val_false_negatives: 49.0000 - val_false_positives: 80.0000
Epoch 8/60
130/130 [==============================] - ETA: 0s - loss: 0.2713 - binary_accuracy: 0.8917 - auc: 0.9532 - mcc_metric: 0.7714 - precision: 0.8085 - true_negatives: 2354.0000 - true_positives: 1343.0000 - false_negatives: 131.0000 - false_positives: 318.0000
Epoch 00008: val_auc did not improve from 0.91032
130/130 [==============================] - 15s 114ms/step - loss: 0.2713 - binary_accuracy: 0.8917 - auc: 0.9532 - mcc_metric: 0.7714 - precision: 0.8085 - true_negatives: 23

In [17]:
model.load_weights(save(34))
model.evaluate(np.array(xtest), np.array(ytest))

39/39 [==============================] - 2s 45ms/step - loss: 0.3966 - binary_accuracy: 0.8298 - auc: 0.9023 - mcc_metric: 0.6357 - precision: 0.7335 - true_negatives: 669.0000 - true_positives: 355.0000 - false_negatives: 81.0000 - false_positives: 129.0000


[0.3966178894042969,
 0.8298217058181763,
 0.902349054813385,
 0.6356543302536011,
 0.7334710955619812,
 669.0,
 355.0,
 81.0,
 129.0]

In [18]:
model.evaluate(np.array(xvalid), np.array(yvalid))

25/25 [==============================] - 1s 47ms/step - loss: 0.3721 - binary_accuracy: 0.8367 - auc: 0.9103 - mcc_metric: 0.6496 - precision: 0.7324 - true_negatives: 442.0000 - true_positives: 219.0000 - false_negatives: 49.0000 - false_positives: 80.0000


[0.3720516860485077,
 0.8367088437080383,
 0.9103155732154846,
 0.6496498584747314,
 0.7324414849281311,
 442.0,
 219.0,
 49.0,
 80.0]